This notebook should be run from the Assignment_3 directory. <br> All input datasets are located in the Data/ subdirectory relative to this file. <br> Please see the README.md file for an overview of the project, data sources, and instructions for reproducing the visualizations.

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt


In [ ]:

# Load the local GeoJSON file
geoData = gpd.read_file("Data/City Wards Data - 4326.geojson")

# Check what you loaded
# Explore both datasets
print("WARD DATA COLUMNS")
print(geoData.columns)
print("WARD DATA SAMPLE")
print(geoData.head())

# Quick plot to verify data loaded correctly
geoData.plot(figsize=(12, 10))
plt.title('Toronto Wards')
plt.show()

In [ ]:

# Load library branch data
library_data = pd.read_csv("Data/tpl-branch-general-information-2023.csv")

print("LIBRARY DATA COLUMNS")
print(library_data.columns)
print("LIBRARY DATA SAMPLE")
print(library_data.head())


In [ ]:
#Visual inspection of data dictionary suggests AREA_SHORT_CODE and WardNo are the key columns to join on
# Check AREA_SHORT_CODE = 1 in geoData
print("=== geoData where AREA_SHORT_CODE = 1 ===")
print(geoData[geoData['AREA_SHORT_CODE'] == '01'][['AREA_SHORT_CODE', 'AREA_NAME']])

# Check WardNo = 1 in library_data
print("\n=== library_data where WardNo = 1 ===")
print(library_data[library_data['WardNo'] == 1][['WardNo', 'WardName', 'BranchName']])


In [ ]:
# Check for missing WardNo values
print("Missing WardNo values:")
print(library_data[library_data['WardNo'].isna()][['BranchName', 'WardNo', 'WardName']])


In [ ]:
# There are virtual and mobile branches - will remove these from the dataset

library_data_clean = library_data.dropna(subset=['WardNo'])

In [ ]:
# Convert WardNo to match AREA_SHORT_CODE format
library_data_clean['WardNo_formatted'] = library_data_clean['WardNo'].astype(int).astype(str).str.zfill(2)
# See https://stackoverflow.com/questions/41749681/converting-numbers-to-particular-string-format-in-a-pandas-dataframe

# Merge library data and geospatial datasets, keeping all library branches as separate rows
fullData = geoData.merge(
    library_data_clean,
    left_on='AREA_SHORT_CODE',  # '01', '02', etc.
    right_on='WardNo_formatted'  # now also '01', '02', etc.
)

fullData.head(10)

print(fullData.columns)
print(fullData.shape)

In [ ]:
# Check data type of SquareFootage
print(fullData['SquareFootage'].dtype)

# Convert SquareFootage to numeric
fullData['SquareFootage'] = pd.to_numeric(fullData['SquareFootage'])


In [ ]:
# Step 1: Group by ward and sum the square footage
# See https://realpython.com/pandas-groupby/ on using groupby and agg in pandas
ward_sqft = fullData.groupby('AREA_SHORT_CODE').agg({
    'SquareFootage': 'sum',
    'AREA_NAME': 'first',  
    'geometry': 'first'     
}).reset_index()

# Convert back to GeoDataFrame
# See on creating geodataframe https://geopandas.org/en/stable/gallery/create_geopandas_from_pandas.html
ward_sqft = gpd.GeoDataFrame(ward_sqft, geometry='geometry')

print("=== Total Square Footage by Ward ===")
print(ward_sqft[['AREA_SHORT_CODE', 'AREA_NAME', 'SquareFootage']].sort_values('SquareFootage', ascending=False))

print(ward_sqft['SquareFootage'].describe())



In [ ]:
print("=== Square Footage Statistics ===")
print(ward_sqft['SquareFootage'].describe())

In [ ]:
# Provide the quintile cut-off values of the total square footage by ward
quintile_bins = pd.qcut(ward_sqft['SquareFootage'], q=5, retbins=True)

print("Quintile cut-offs:")
print(quintile_bins[1])

In [ ]:
# Find the reference libraries from the original clean data
ref_libraries = fullData[fullData['BranchName'].isin(['Toronto Reference Library', 'North York Central Library'])]


# Show the coordinates
print(ref_libraries[['BranchName', 'Lat', 'Long']])

In [ ]:
# Create quintiles of square footage by ward for labelling and plotting
ward_sqft['sqft_quantile'] = pd.qcut(ward_sqft['SquareFootage'], 
                                      q=5, 
                                      labels=['Q1 ≤ 35,408 sq ft', 
                                                'Q2 35,409 – 41,481 sq ft', 
                                                'Q3 41,482 – 52,484 sq ft', 
                                                'Q4 52,485 – 70,227 sq ft', 
                                                'Q5 ≥ 70,228 sq ft'])

# Create figure and axes 
fig, ax = plt.subplots(figsize=(15, 12))

# Plot the choropleth
ward_sqft.plot(column='sqft_quantile',
               ax=ax,
               legend=True,
               cmap='viridis', 
               edgecolor='black',
               linewidth=0.5,
               categorical=True,
               legend_kwds={'loc':'lower right'})   # See geoPandas documentation for legend_kwds options
                                                    # https://geopandas.org/en/stable/docs/user_guide/mapping.html
                   

# Add Toronto Reference Library marker 
bold_font = {'fontweight': 'bold'}
ax.scatter(-79.3729, 43.671737,  # Using coordinates from above code
           marker='*', 
           s=200,
           color='blue', 
           edgecolor='white',
           linewidth=2)

# Add Toronto Reference Library label
ax.text(-79.3729, 43.671737 + 0.01,  
        'Toronto Reference Library',
        fontsize=8,
        ha='center',
        color='blue',
        fontdict=bold_font)

# Add North York Central Library marker
ax.scatter(-79.4114, 43.768078,  # Using coordinates from above code 
           marker='*', 
           s=200,
           color='blue', 
           edgecolor='white',
           linewidth=2)

# Add North York Central Library label 
ax.text(-79.4114, 43.768078 + 0.01,  
        'North York Central Library',
        fontsize=8,
        ha='center',
        color='blue',
        fontdict=bold_font)


# Add main title 
ax.set_title('Toronto Public Library Space by Ward (2023): Quintiles of Total Square Footage', 
             fontsize=24,
             pad=1)  

# Add subtitle below title 
ax.text(0.5, 0.98, 'Wards grouped into quintiles due to skew from Reference libraries',
        transform=ax.transAxes,
        fontsize=14,
        ha='right',
        va='top')  

# Add data source footnote
ax.text(0.2, 0.02, 'Data source: Toronto Open Data Portal (Toronto Public Library – Branch Information)',
        transform=ax.transAxes,
        fontsize=10,
        ha='center',
        va='bottom')

ax.axis('off')
fig.tight_layout()

# Show the plot
fig.show()

# Saving the figure
plt.savefig(
    "Visualization_2_Python.png",
    dpi=300,
    bbox_inches="tight"
)
plt.close()

In [ ]:
# Creating Excel dataset
# Count number of branches per ward
branches_per_ward = fullData.groupby('AREA_SHORT_CODE').size().reset_index(name='branch_count')

# Merge branch counts
ward_sqft = ward_sqft.merge(branches_per_ward, on='AREA_SHORT_CODE', how='left')

print("=== Ward Square Footage with Branch Counts ===")
print(ward_sqft[['AREA_SHORT_CODE', 'AREA_NAME', 'SquareFootage', 'branch_count']].sort_values('SquareFootage', ascending=False))   

In [ ]:
# Exporting to Excel
ward_sqft_excel = ward_sqft[['AREA_SHORT_CODE', 'AREA_NAME', 'SquareFootage','branch_count']].copy()

ward_sqft.to_excel("Data/toronto_library_sqft_by_ward.xlsx",index=False)